In [0]:
import numpy as np
import random
import math
from random import randint
from ortools.sat.python import cp_model

In [0]:
#Initial Parameters
n_PPE=700
n_hospitals=5
n_staff=[50,55,65,72,45]

In [0]:
#Assigning severity Randomly
n_patients=134
n_patients_in=[30,20,35,34,15]
patients_severity=[]

for i in range(n_hospitals):
  temp=[]
  for j in range(0,n_patients_in[i]):
    temp.append(random.randint(1,5))
  patients_severity.append(temp)

In [0]:
#Averaging the severity in all the hospitals for finding the most sever hospital
avg_rows = [math.ceil(sum(x)/len(x)) for x in patients_severity]

In [5]:
print(avg_rows)

[3, 4, 3, 3, 4]


In [0]:
model = cp_model.CpModel()
x = {}

In [0]:
#Creating the variable
for i in range(n_hospitals):
  for j in range(n_staff[i]):
    for k in range(n_PPE):
      x[(i,j,k)] = model.NewBoolVar("x(%d,%d,%d)" % (i,j,k))

In [0]:
#All staff should be given at least one PPE
for i in range(n_hospitals):
  for j in range(n_staff[i]):
    model.Add(sum(x[(i,j,k)] for k in range(n_PPE)) >= 1)

In [0]:
#No two staff can share a single PPE
for k in range(n_PPE):
  inner_sum=[]
  for i in range(n_hospitals):
    inner_sum.append(sum(x[(i,j,k)] for j in range(n_staff[i])))
  model.Add(sum(inner_sum) == 1)

In [0]:
#No PPE should remain unallocated
for k in range(n_PPE):
    inner_sum = []
    for i in range(n_hospitals):
        inner_sum.append(sum(x[(i,j,k)] for j in range(n_staff[i]))) 
    model.Add(sum(inner_sum) >= 1)

In [0]:
#Soft constraint to maximize the PPE allocated based on number of staffs and severe patients
soft_csts=[]
#Randomly Assigned gained factors. The result changes according to the factors.
gain_severity=10
gain_staff=50
for i in range(n_hospitals):
    for j in range(n_staff[i]):
        for k in range(n_PPE):            
            soft_csts.append((gain_staff*n_staff[i]+gain_severity*avg_rows[i]) * x[(i,j,k)])

In [0]:
model.Maximize(sum(soft_csts))

In [0]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 300.0
status = solver.Solve(model)

In [14]:
for i in range(n_hospitals):
  for j in range(n_staff[i]):
    for k in range(n_PPE):
      if solver.Value(x[i,j,k])==1:
        print("Kit : " + str(k) + " Hospital : " + str(i) + " Staff : " + str(j))

Kit : 554 Hospital : 0 Staff : 0
Kit : 166 Hospital : 0 Staff : 1
Kit : 116 Hospital : 0 Staff : 2
Kit : 267 Hospital : 0 Staff : 3
Kit : 117 Hospital : 0 Staff : 4
Kit : 118 Hospital : 0 Staff : 5
Kit : 365 Hospital : 0 Staff : 6
Kit : 4 Hospital : 0 Staff : 7
Kit : 25 Hospital : 0 Staff : 8
Kit : 154 Hospital : 0 Staff : 9
Kit : 583 Hospital : 0 Staff : 10
Kit : 188 Hospital : 0 Staff : 11
Kit : 581 Hospital : 0 Staff : 12
Kit : 119 Hospital : 0 Staff : 13
Kit : 146 Hospital : 0 Staff : 14
Kit : 636 Hospital : 0 Staff : 15
Kit : 679 Hospital : 0 Staff : 16
Kit : 487 Hospital : 0 Staff : 17
Kit : 568 Hospital : 0 Staff : 18
Kit : 324 Hospital : 0 Staff : 19
Kit : 15 Hospital : 0 Staff : 20
Kit : 349 Hospital : 0 Staff : 21
Kit : 699 Hospital : 0 Staff : 22
Kit : 683 Hospital : 0 Staff : 23
Kit : 428 Hospital : 0 Staff : 24
Kit : 506 Hospital : 0 Staff : 25
Kit : 99 Hospital : 0 Staff : 26
Kit : 29 Hospital : 0 Staff : 27
Kit : 340 Hospital : 0 Staff : 28
Kit : 470 Hospital : 0 Staff :

In [15]:
for i in range(n_hospitals):
  count=0
  for j in range(n_staff[i]):
    for k in range(n_PPE):
      if solver.Value(x[(i,j,k)])==1:
        count=count+1
  print("Hospital --> " + str(i) + " PPEs : "+ str(count))

Hospital --> 0 PPEs : 50
Hospital --> 1 PPEs : 55
Hospital --> 2 PPEs : 65
Hospital --> 3 PPEs : 485
Hospital --> 4 PPEs : 45
